In [1]:
import numpy as np
import pandas as pd
import csv
import random
import math

# Инициализация

# Считывание данных из файла и их подготовка

In [3]:
# количество признаков входного слоя
feature_count = 4

df = pd.read_csv('data/iris.csv', sep=',')

# подготовленный датасет
train_data = []
# датасет для тестирования
test_data = []

classes = df['variety'].unique()

df = df.sample(frac=1).reset_index(drop=True)

for i in range(len(df)) : 

    features = np.zeros((feature_count))
    features[0] = df.loc[i, "sepal.length"]
    features[1] = df.loc[i, "sepal.width"]
    features[2] = df.loc[i, "petal.length"]
    features[3] = df.loc[i, "petal.width"]
    
    # Нормализация входного вектора
    norm_features = np.zeros((feature_count))
    for j in range(len(features)):
        sum = 0 
        for k in range(len(features)):
            sum += features[k] ** 2
        norm_features[j] = features[j] / math.sqrt(sum)
    
    temp_data = dict(features = norm_features, result = np.where(classes == df.loc[i, "variety"])[0].item(0))
    if (len(train_data) / len(df)) < 0.9:
        train_data.append(temp_data)
    else:
        test_data.append(temp_data)
    print(temp_data)

{'features': array([0.75519285, 0.33928954, 0.53629637, 0.16417236]), 'result': 1}
{'features': array([0.69385414, 0.29574111, 0.63698085, 0.15924521]), 'result': 2}
{'features': array([0.69595601, 0.3427843 , 0.59208198, 0.21813547]), 'result': 2}
{'features': array([0.80033301, 0.56023311, 0.20808658, 0.04801998]), 'result': 0}
{'features': array([0.86093857, 0.44003527, 0.24871559, 0.0573959 ]), 'result': 0}
{'features': array([0.72337118, 0.34195729, 0.57869695, 0.15782644]), 'result': 1}
{'features': array([0.67017484, 0.36168166, 0.59571097, 0.2553047 ]), 'result': 2}
{'features': array([0.79524064, 0.54144043, 0.27072022, 0.03384003]), 'result': 0}
{'features': array([0.77867447, 0.59462414, 0.19820805, 0.02831544]), 'result': 0}
{'features': array([0.70610474, 0.3258945 , 0.59747324, 0.1955367 ]), 'result': 2}
{'features': array([0.790965 , 0.5694948, 0.2214702, 0.0316386]), 'result': 0}
{'features': array([0.69299099, 0.34199555, 0.60299216, 0.19799743]), 'result': 2}
{'featur

# Обучение

## Кохонен

In [4]:
def getKohonenWinnerIndex(x, neuron_counts, w, s):
    result = 0
    max_net = -1
    for i in range(neuron_counts):
        net = 0
        for j in range(len(x)):
            net += x[j] * w[j][i] * s[i]
        if (net > max_net):
            result = i
            max_net = net
    return result

In [5]:
# скорость обучения слоя Кохонена
theta = 0.7
# скорость обучения слоя Гроссберга
beta = 0.1
#количество нейронов выходного слоя (слоя Гроссберга) = количество проффессий 
grossberg_neuron_counts = len(classes)
# количество нейронов кохонена
kohonen_neuron_count = 9
# штрафы кластеров
s_w = np.ones((kohonen_neuron_count))
# количество эпох
epoch_count = 50

#инициализация начальных весов слоя Кохонена kohonen_w[x][y]
# x - индекс нейрона входного слоя
# y - индекс нейрона слоя Кохонена
kohonen_w = np.zeros((feature_count, kohonen_neuron_count))
for i in range(feature_count):
    for j in range(kohonen_neuron_count):
        kohonen_w[i, j] = random.uniform(0.01, 0.99)

# Инициализация начальных весов слоя Гроссберга grossberg_w[x][y]
# x - индекс нейрона слоя Кохонена
# y - индекс нейрона слоя Гроссберга
grossberg_w = np.zeros((kohonen_neuron_count, grossberg_neuron_counts))
for i in range(kohonen_neuron_count):
    for j in range(grossberg_neuron_counts):
        grossberg_w[i, j] = random.uniform(0, 1)

step = epoch_count* len(train_data)
# Обучение на подготовленном датасете
while step >= 0:
    
    blossom = train_data[random.randint(0, len(train_data) - 1)]
    
    # Определение нейрона победителя
    winner_index = getKohonenWinnerIndex(blossom['features'], kohonen_neuron_count, kohonen_w, s_w)
    s_w[winner_index] *= 0.9
    
    # Корректировка весов нейрона победителя на слое Кохонена
    for i in range(feature_count):
        kohonen_w[i][winner_index] += theta * (blossom['features'][i] - kohonen_w[i][winner_index])

    # Корректировка весов для нейронов слоя Гроссберга, соединенных с нейроном победителем слоя Кохонена
    for i in range(grossberg_neuron_counts):
        if i == blossom['result']:
            grossberg_w[winner_index][i] += beta * (1 - grossberg_w[winner_index][i])
        else:
            grossberg_w[winner_index][i] += beta * (0 - grossberg_w[winner_index][i])

    # Изменение скорости обучения
    theta, beta = theta - (theta * 0.0003), beta - (beta * 0.0003)  
    step -= 1

# Тестирование

In [6]:
accuracy = 0
for blossom in test_data:
    
    print('Класс: ' + classes[blossom['result']])
    print(blossom['features'])
    print('Результаты нейросети:')
    
    # Определение нейрона победителя
    winner_index = getKohonenWinnerIndex(blossom['features'], kohonen_neuron_count, kohonen_w, s_w)
    best_result = 0.0
    best_index = 0

    for i in range(grossberg_neuron_counts):
        if (grossberg_w[winner_index][i] > best_result):
            best_result = grossberg_w[winner_index][i]
            best_indexe = i
        print(classes[i] + ': ' + str(grossberg_w[winner_index][i] * 100) + '%')
    
    if (blossom['result'] == best_index):
        accuracy += 1
    print('--------------------------------------------------')
print ('Точность: ' + str((accuracy / len(test_data)) * 100) + '%')

Класс: Versicolor
[0.73260391 0.36029701 0.55245541 0.1681386 ]
Результаты нейросети:
Setosa: 0.05048271638029879%
Versicolor: 70.97021596025162%
Virginica: 28.979301323368485%
--------------------------------------------------
Класс: Setosa
[0.80846584 0.52213419 0.26948861 0.03368608]
Результаты нейросети:
Setosa: 92.78318835898679%
Versicolor: 5.109361213421994%
Virginica: 2.1074504275914547%
--------------------------------------------------
Класс: Virginica
[0.69052512 0.32145135 0.60718588 0.22620651]
Результаты нейросети:
Setosa: 1.6932025778410371%
Versicolor: 16.104494360573522%
Virginica: 82.20230306158547%
--------------------------------------------------
Класс: Virginica
[0.67054118 0.34211284 0.61580312 0.23263673]
Результаты нейросети:
Setosa: 1.6932025778410371%
Versicolor: 16.104494360573522%
Virginica: 82.20230306158547%
--------------------------------------------------
Класс: Setosa
[0.77729093 0.57915795 0.24385598 0.030482  ]
Результаты нейросети:
Setosa: 92.78318